In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

# Daten einlesen
df = pd.read_csv('../data/kostenvorhersage.csv')

# Daten aufbereiten
X = df.drop('Kosten', axis=1)
y = df['Kosten']

# Daten splitten
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Daten skalieren
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Daten reshapen
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Model erstellen
model = Sequential()

# Input Layer
model.add(Conv2D(64, (3,3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Hidden Layer
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Output Layer
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

# Model kompilieren
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Tensorboard
NAME = "Kostenvorhersage-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# Model trainieren
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          validation_split=0.3,
          callbacks=[tensorboard])

# Model evaluieren
val_loss, val_acc = model.evaluate(X_test, y_test)
print(val_loss, val_acc)

# Model speichern
model.save('../models/kostenvorhersage.model')